In [1]:
# from google.colab import auth
# auth.authenticate_user()
# import gspread
# from oauth2client.client import GoogleCredentials
# gc = gspread.authorize(GoogleCredentials.get_application_default())

In [2]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
  !pip install Pytrends
  !pip install pandas

  Created wheel for Pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16126 sha256=d124b7f5fce5a346f33135bd1404f6298737c7e4a18ee975173e66c300af9856
  Stored in directory: /root/.cache/pip/wheels/07/6f/5c/8174f98dec1bfbc7d5da4092854afcbcff4b26c3d9b66b5183
Successfully built Pytrends


In [10]:
## https://www.honchosearch.com/blog/seo/how-to-use-python-pytrends-to-automate-google-trends-data/

from pytrends.request import TrendReq
import pandas as pd
import time
import random
startTime = time.time()
pytrend = TrendReq(hl='en-GB', tz=360)
df = wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1-bEjtLyLxjwo5VXTmvs1mgXSelsz9SBgU_R-7rPIE_A/edit?usp=sharing')
geo1 = 'GB' # 'GB' #GLOBAL doesnt work, use "" for global
kwrds_location = "_world" # "_UK"
topics = [f"keyword_finance{kwrds_location}", f"keyword_learning{kwrds_location}", f"keyword_jobs{kwrds_location}", f"keyword_research{kwrds_location}", f"keyword_support{kwrds_location}", f"keyword_gaming{kwrds_location}", 
          f"keyword_realestate{kwrds_location}", f"keyword_virtual{kwrds_location}", f"keyword_leisure{kwrds_location}", f"keyword_mentalhealth{kwrds_location}", f"keyword_sustainability{kwrds_location}", f"keyword_used{kwrds_location}"]

# store data here
dfs = []

# loop through topics and collect data
for topic in topics:
  print("scraping", topic)
  sheet = wb.worksheet(topic)
  df2 = sheet.col_values(5)
  d_from = sheet.acell('B7').value
  d_to = sheet.acell('B8').value
  # geo1 = geo #'GLOBAL' or 'GB' # geo1 = sheet.acell('B10').value 
  dataset = []
  for x in range(0,len(df2)):
    keywords = [df2[x]]
    try:
      pytrend.build_payload(
        kw_list=keywords,
        cat=0,
        timeframe= str(d_from + " " + d_to)
        ,
        geo = geo1)
    except:
      print("429 error, moving on, sleeping..")
      time.sleep(random.randint(10,30))
      
    data = pytrend.interest_over_time()
    if not data.empty:
      data = data.drop(labels=['isPartial'], axis='columns')
      dataset.append(data)
 
  result = pd.concat(dataset, axis=1)
  result["topic"] = topic.strip("keyword_")
  result[topic.strip("keyword_")+"_mean"] = result.mean(axis=1)
  result_cl = result.reset_index().melt(id_vars=["date", "topic"])
  dfs.append(result_cl)
  # print("finished scraping", topic)
  time.sleep(random.randint(10,60))

trends_all_world = pd.concat(dfs)

scraping keyword_finance_world


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


scraping keyword_learning_world
scraping keyword_jobs_world
scraping keyword_research_world
scraping keyword_support_world
scraping keyword_gaming_world
429 error, moving on, sleeping..


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


scraping keyword_realestate_world
429 error, moving on, sleeping..
scraping keyword_virtual_world
scraping keyword_leisure_world
429 error, moving on, sleeping..
scraping keyword_mentalhealth_world
scraping keyword_sustainability_world
scraping keyword_used_world


In [8]:
# trends_all_world

In [11]:
len(dfs)

12

In [12]:
trends_all_world

,date,topic,variable,value
0,2017-01-01,finance_worl,finance,71.00000
1,2017-01-08,finance_worl,finance,66.00000
2,2017-01-15,finance_worl,finance,66.00000
3,2017-01-22,finance_worl,finance,68.00000
4,2017-01-29,finance_worl,finance,66.00000
...,...,...,...,...
8608,2021-11-28,used_worl,used_worl_mean,44.56250
8609,2021-12-05,used_worl,used_worl_mean,39.62500
8610,2021-12-12,used_worl,used_worl_mean,39.40625
8611,2021-12-19,used_worl,used_worl_mean,38.62500


In [15]:
trends_all_world.head()

,date,topic,variable,value
0,2017-01-01,finance_worl,finance,71.0
1,2017-01-08,finance_worl,finance,66.0
2,2017-01-15,finance_worl,finance,66.0
3,2017-01-22,finance_worl,finance,68.0
4,2017-01-29,finance_worl,finance,66.0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

trends_all_world.to_csv('trends_all_GB_world_09042022.csv', encoding = 'utf-8-sig') 
files.download('trends_all_GB_world_09042022.csv')

<IPython.core.display.Javascript object>

In [37]:
drive.mount('/content/drive')
path = '/content/drive/My drive/Project_GoogleSearchTrends/worldData/trends_all_world_08042022.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  trends_all_world.to_csv(f)

In [ ]:
#trends_all.to_csv('trends_all.csv')
#!cp trends_all.csv "/content/drive/My drive/EndersAnalysis/Project_GoogleSearchTrends/UKData"
#executionTime = (time.time() - startTime)
#print('Execution time in sec.: ' + str(executionTime))

cp: cannot create regular file '/content/drive/My drive/EndersAnalysis/Project_GoogleSearchTrends/UKData': No such file or directory
Execution time in sec.: 491.97405982017517
